# Frequency Distribution

In this tutorial, we will learn how to organize data in frequency and groiuped frequency tables. Frequency tables allow us to transform large and incomprehensible amounts of data to a format we can understand.
